In [ ]:
#this command might be relevant ony on GPU nodes. But the notebook will run on CPU as well
#If running on CPU remove the command wherever it appears
!nvidia-smi

Mon Dec 23 03:51:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.8/265.8 MB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.46.0.6-py2.py3-none-any.whl size=265859786 sha256=0ca365e0ff7684db0c3249f16e7ee4df1ba204796d00e2f6cb8580bad668669a
  Stored in directory: /root/.cache/pip/wheels/0c/95/eb/b449c73f478dbc4557b80170cf7f2aa6db15862877e9a63536
Successfully built h2o


In [ ]:
import pandas as pd
import h2o
from h2o.estimators import H2OGradientBoostingEstimator
from sklearn.metrics import r2_score

In [ ]:
data1=pd.read_csv('/content/sample_data/diabetes_prediction_dataset.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/content/sample_data/diabetes_prediction_dataset.csv'

In [ ]:
data1['diabetes']=data1['diabetes'].replace(1,"yes")

In [ ]:
data1['diabetes']=data1['diabetes'].replace(0,"no")

In [ ]:
data1.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,no
1,Female,54.0,0,0,No Info,27.32,6.6,80,no
2,Male,28.0,0,0,never,27.32,5.7,158,no
3,Female,36.0,0,0,current,23.45,5.0,155,no
4,Male,76.0,1,1,current,20.14,4.8,155,no


In [ ]:
data1.isnull().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

In [ ]:
data=data

In [ ]:
import h2o
from h2o.automl import H2OAutoML
h2o.init(nthreads=-1,max_mem_size=12)
data = data1
train=h2o.H2OFrame(data)
xc=list(data.columns)
yc="diabetes"
aml=H2OAutoML(max_models=10,seed=1)
aml.train(x=xc,y=yc,training_frame=train)

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,32 mins 34 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_unknownUser_5jx6xb
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.992 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


key,value
Stacking strategy,cross_validation
Number of base models (used / total),8/10
# GBM base models (used / total),3/4
# XGBoost base models (used / total),3/3
# DRF base models (used / total),1/2
# GLM base models (used / total),1/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5
Metalearner fold_column,None


In [ ]:
!pip install PyCaret
from pycaret.regression import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.9/485.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 60.5

In [ ]:
import pandas as pd

data = pd.read_csv('/content/sample_data/diabetes_prediction_dataset.csv')
setup(data, target='diabetes',train_size=0.8)
best_model = compare_models(sort='MAE')
lr_model = create_model('lr')
evaluate_model(lr_model)
tuned_lr_model = tune_model(lr_model)
final_model = finalize_model(tuned_lr_model)
predictions = predict_model(final_model, data=data)


,Description,Value
0,Session id,8193
1,Target,diabetes
2,Target type,Regression
3,Original data shape,"(100000, 9)"
4,Transformed data shape,"(100000, 16)"
5,Transformed train set shape,"(80000, 16)"
6,Transformed test set shape,"(20000, 16)"
7,Numeric features,6
8,Categorical features,2
9,Preprocess,True


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:50:38
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Lasso Least Angle Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,0.1483,0.0630,0.2511,0.1940,0.1759,0.7726,0.3420
en,Elastic Net,0.1520,0.0613,0.2475,0.2165,0.1750,0.7411,0.3530
lr,Linear Regression,0.1540,0.0506,0.2249,0.3531,0.1647,0.5913,1.0550
ridge,Ridge Regression,0.1540,0.0506,0.2249,0.3531,0.1647,0.5913,0.4920
lar,Least Angle Regression,0.1540,0.0506,0.2249,0.3531,0.1647,0.5913,0.2000


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
data.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [ ]:
!pip install gradio


import gradio as gr



def predict(gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes):
  pred = predict_model(final_model,data=pd.DataFrame({'gender':[gender],'age':[age],'hypertension':[hypertension],'heart_disease':[heart_disease],'smoking_history':[smoking_history],'bmi':[bmi], 'casual':[casual], 'HbA1c_level':[HbA1c_level],'blood_glucose_level':[blood_glucose_level]},index=[0]))
  return pred

app=gr.Interface(fn=predict,inputs=["number","number","number","number","number","number","number","number"],outputs=gr.Textbox(),title="h2o Model")
app.launch(debug=True)

/usr/local/lib/python3.10/dist-packages/gradio/utils.py:915: UserWarning: Expected 9 arguments for function <function predict at 0x78b4b6e90670>, received 8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:919: UserWarning: Expected at least 9 arguments for function <function predict at 0x78b4b6e90670>, received 8.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://9f23b8b1b18a758f77.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import h2o
import gradio as gr

# Start H2O
h2o.init()

# Load your trained H2O model
model = h2o.load_model("/content/sample_data/diabetes_prediction_dataset.csv")

# Define a function to make predictions using your H2O model
def predict(input_data):
    # Create an H2O frame from the input data
    input_frame = h2o.H2OFrame(input_data)
    # Use your trained H2O model to make predictions
    predictions = model.predict(input_frame)
    return predictions.as_data_frame().values.flatten().tolist()

# Create a Gradio interface
input_textbox = gr.inputs.Textbox(lines=5, label="Input Data")
output_text = gr.outputs.Textbox(label="Predictions")

interface = gr.Interface(fn=predict, inputs=input_textbox, outputs=output_text, title="H2O Model with Gradio")
interface.launch()


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,43 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_unknownUser_5jx6xb
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Missing magic number 0x1CED at stream start
  Request: POST /99/Models.bin/
    data: {'dir': '/content/sample_data/diabetes_prediction_dataset.csv'}
